In [ ]:
import pandas as pd 
df = pd.read_csv('/content/spam_ham_dataset.csv') 
df.drop(df.filter(regex="Unname"),axis=1, inplace=True)
from nltk.tokenize import RegexpTokenizer
def clean_str(string, reg = RegexpTokenizer(r'[a-z]+')): 
    # Clean a string with RegexpTokenizer
    string = string.lower() 
    tokens = reg.tokenize(string) 
    return " ".join(tokens) 
df['text_clean'] = df['text'].apply(lambda string: clean_str(string))
df_spam = df[df['label']=='spam']
df_ham = df[df['label']=='ham'] 
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.drop(['text'], axis=1,inplace=True)
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df_balanced['text_clean'],df_balanced['label_num'], stratify=df_balanced['label_num'])
import tensorflow as tf 
import tensorflow_hub as hub 
!pip install -U tensorflow-text==2.6.0 
import tensorflow_text as text 
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
# building sequencial model # Bert layers 
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers 
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output']) 
#DropoutLayer - Dropout Regularization - some might beb biased with input features toh overfiiting remove
#pooledEncoding-shape-no. of txt, words,word embedding vector size
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])
METRICS = [ 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
] 
model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=METRICS) 
from keras.callbacks import ModelCheckpoint,EarlyStopping
mc=ModelCheckpoint(filepath="/content/saved_model/best_model.h5", # Where best file will be saved
                                              # best model would go here
                   monitor="accuracy",
                   verbose=1, 
                   save_best_only=True) 
es=EarlyStopping(monitor="accuracy", 
                 min_delta=0.01, 
                 patience=5, # No of time it should wait if accuracy doesnt increase
                 verbose=1)
cb=[mc,es]
model.fit(X_train, y_train, epochs=30,callbacks=cb)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Epoch 1/30
71/71 [==============================] - 892s 12s/step - loss: 0.6396 - accuracy: 0.6250 - precision: 0.6188 - recall: 0.6512

Epoch 00001: accuracy improved from -inf to 0.62500, saving model to /content/saved_model/best_model.h5
Epoch 2/30
71/71 [==============================] - 874s 12s/step - loss: 0.5759 - accuracy: 0.6899 - precision: 0.6993 - recall: 0.6664

Epoch 00002: accuracy improved from 0.62500 to 0.68995, saving model to /content/saved_model/best_model.h5
Epoch 3/30
71/71 [==============================] - 873s 12s/step - loss: 0.5343 - accuracy: 0.7469 - precision: 0.7520 - recall: 0.7367

Epoch 00003: accuracy improved from 0.68995 to 0.74689, saving model to /content/saved_model/best_model.h5
Epoch 4/30
71/71 [==============================] - 878s 12s/step - loss: 0.5053 - accuracy: 0.7745 - precision: 0.7777 - recall: 0.7687

Epoch 00004: accuracy improved 

In [ ]:
model.evaluate(X_test, y_test)

24/24 [==============================] - 297s 12s/step - loss: 0.2727 - accuracy: 0.9173 - precision: 0.9563 - recall: 0.8747


[0.2726881802082062, 0.9173333048820496, 0.9562682509422302, 0.874666690826416]

In [ ]:
y_predicted = model.predict(X_test)
y_predicted 

array([[0.8977351 ],
       [0.9648032 ],
       [0.24423906],
       [0.09989417],
       [0.5119101 ],
       [0.0125874 ],
       [0.91329   ],
       [0.57473534],
       [0.5875761 ],
       [0.8790566 ],
       [0.34818852],
       [0.7017441 ],
       [0.92723805],
       [0.3718269 ],
       [0.1117624 ],
       [0.10903108],
       [0.6848657 ],
       [0.96549946],
       [0.95561606],
       [0.6706395 ],
       [0.02034804],
       [0.08365983],
       [0.04314968],
       [0.8188089 ],
       [0.10134268],
       [0.05907387],
       [0.86684626],
       [0.12478039],
       [0.04052982],
       [0.02739891],
       [0.04906222],
       [0.07826823],
       [0.97009426],
       [0.07033205],
       [0.10280964],
       [0.2905305 ],
       [0.2626843 ],
       [0.5709429 ],
       [0.52675796],
       [0.06817579],
       [0.11778811],
       [0.9802563 ],
       [0.9351923 ],
       [0.12493089],
       [0.2117132 ],
       [0.7470538 ],
       [0.08679736],
       [0.825

In [ ]:
import numpy as np
def predict(mail):
     y_predicted = model.predict(X_test)
     y_predicted = y_predicted.flatten()
     y_predicted = np.where(y_predicted > 0.5, 1, 0)

In [ ]:
y_predicted

array([[0.8977351 ],
       [0.9648032 ],
       [0.24423906],
       [0.09989417],
       [0.5119101 ],
       [0.0125874 ],
       [0.91329   ],
       [0.57473534],
       [0.5875761 ],
       [0.8790566 ],
       [0.34818852],
       [0.7017441 ],
       [0.92723805],
       [0.3718269 ],
       [0.1117624 ],
       [0.10903108],
       [0.6848657 ],
       [0.96549946],
       [0.95561606],
       [0.6706395 ],
       [0.02034804],
       [0.08365983],
       [0.04314968],
       [0.8188089 ],
       [0.10134268],
       [0.05907387],
       [0.86684626],
       [0.12478039],
       [0.04052982],
       [0.02739891],
       [0.04906222],
       [0.07826823],
       [0.97009426],
       [0.07033205],
       [0.10280964],
       [0.2905305 ],
       [0.2626843 ],
       [0.5709429 ],
       [0.52675796],
       [0.06817579],
       [0.11778811],
       [0.9802563 ],
       [0.9351923 ],
       [0.12493089],
       [0.2117132 ],
       [0.7470538 ],
       [0.08679736],
       [0.825

In [ ]:
def predict(mail):
     y_predicted = model.predict(X_test)
     y_predicted = y_predicted.flatten()
     y_predicted = np.where(y_predicted > 0.5, 1, 0)
     if abs(y_predicted).any==1: 
      print ("Spam") 
     else:
      print ("Ham") 

Now use saved model for prediction

There is one factor, which you must to consider. You need to change the learning phase, before converting. It's super important, when you have Dropout or Batch Normalization. Look at 'Keras model to tflite' or 'Problem after converting keras model into Tensorflow pb'